# Jupyter Dashのデモ（ダッシュボードでグラフとテーブルを表示）

In [30]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [31]:
# JupyterDash他が未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install jupyter_dash
#!pip install --upgrade plotly
#!pip install dash_bootstrap_components

In [32]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [33]:
# matplotlibで日本語を使えるようにする
import japanize_matplotlib

In [34]:
import pandas as pd
import numpy as np

In [35]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from dash import html
from dash import dcc
from dash import dash_table
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash_table import FormatTemplate
from dash.dash_table.Format import Format

## データファイルを読み込む

In [36]:
 df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/都道府県各種指標.xlsx',sheet_name='data')
 df1 = df1.assign(ch_year = df1['年度'].apply(lambda x: str(x)+'年'))
 df1 = df1.assign(id_pref = df1['地域コード']+df1['都道府県'])
 df1.sort_values('年度',inplace=True)
 df1.head(3)

,年度,地域コード,都道府県,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,15歳未満人口（男）,15歳未満人口（女）,15～64歳人口,15～64歳人口（男）,15～64歳人口（女）,65歳以上人口,65歳以上人口（男）,65歳以上人口（女）,出生数,出生数（男）,出生数（女）,合計特殊出生率,死亡数,死亡数（男）,死亡数（女）,転入者数（日本人移動者）,転入者数（日本人移動者）（男）,転入者数（日本人移動者）（女）,転出者数（日本人移動者）,転出者数（日本人移動者）（男）,転出者数（日本人移動者）（女）,婚姻件数,離婚件数,年平均気温,最高気温（日最高気温の月平均の最高値）,最低気温（日最低気温の月平均の最低値）,降水日数（年間）,降水量（年間）,着工建築物数,着工建築物床面積,...,専修学校数,各種学校数,専修学校生徒数,各種学校生徒数,新規求職申込件数（一般）,月間有効求職者数（一般）,月間有効求人数（一般）,充足数（一般）,就職件数（一般）,県内就職件数（一般）,一般旅券発行件数,着工新設住宅戸数,着工新設持家数,着工新設貸家数,着工新設分譲住宅数,着工新設住宅床面積,着工新設持家床面積,着工新設分譲住宅床面積,着工新設貸家床面積,一般病院数,一般診療所数,歯科診療所数,保育所等数,保育所等定員数,保育所等利用待機児童数,保育所等在所児数,保育所等保育士数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）,ch_year,id_pref
563,2007,R47000,沖縄県,1374000,674000,700000,1367000,670000,697000,248000,127000,121000,893000,448000,444000,232000,97000,135000,16588,8434,8154,1.75,9399,5049,4350,25841,13999,11842,28037,15864,12173,8620,3698,23.5,32.5,15.2,133,2816.5,4444,1776041,...,50,41,10068,1830,80876,319727,111065,14554,22140,14367,30953,9614,2133,6448,937,811446,274514,97639,430152,81,792,587,363,28791,1850,32274,5398,220100,51211,21389,20148,6470,6682,9622,25297,10303,17777,51202,2007年,R47000沖縄県
467,2007,R39000,高知県,783000,367000,415000,780000,366000,414000,97000,50000,48000,471000,232000,239000,213000,85000,128000,5717,2947,2770,1.31,9071,4699,4372,10287,5752,4535,14829,8236,6593,3549,1663,17.9,32.6,3.3,103,1859.0,2974,640752,...,28,15,4095,876,42841,177860,67462,9434,10273,9295,12274,3890,1523,1546,812,358174,194601,85801,76774,125,580,364,278,22515,65,20634,2927,284574,65963,20558,20144,9908,9979,10702,35384,14129,25772,72034,2007年,R39000高知県
479,2007,R40000,福岡県,5061000,2395000,2666000,5021000,2375000,2646000,701000,359000,342000,3293000,1607000,1686000,1062000,428000,633000,46393,23842,22551,1.34,43919,22780,21139,107719,60200,47519,109844,62832,47012,29486,11115,18.0,33.7,4.4,100,1195.0,19784,6202007,...,188,53,47101,6246,229234,941985,672335,60424,60762,54311,154713,40943,9945,22911,7828,3243389,1313276,796780,1107395,410,4461,2994,872,92555,322,97325,12745,308853,67751,21043,20774,9772,16896,15624,40340,13307,34726,68618,2007年,R40000福岡県


表示候補の列名のリストを作成する

In [37]:
my_list = df1.columns
to_remove = ['年度','地域コード','都道府県','ch_year','id_pref']
remained_list = [i for i in my_list if i not in to_remove]

Dash - Plotlyでの描画の準備

In [38]:
# Dash表示用の設定：グラフ１（分布グラフ）
default_kpi_1 = '総人口'
plot_type_1 = ['箱ひげ図','散布図','バイオリン']
default_plot_1 = '箱ひげ図'
# Dash表示用の設定：グラフ２（基本グラフ）
default_kpi_2 = '総人口'
plot_type_2 = ['棒グラフ','折れ線グラフ']
default_plot_2 = '棒グラフ'
default_pref_2 = '東京都'
# Dash表示用の設定：グラフ３（データテーブル）
default_kpi_3 = '総人口'

appという「箱」を作る

In [39]:
# JupyterDashの「箱」を作る
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP],
                  meta_tags=[{'name': 'viewport',
                              'content': 'width=device-width, initial-scale=1.0'}]) # レスポンシブ対応のためのメタタグ

グラフ１（分布グラフ）のコンテンツの定義

In [40]:
# ドロップダウンリストを定義する
content1 = dcc.Dropdown(
    id = 'dropdown-for-kpi-1', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 値を選択する列を指定
    value = default_kpi_1 # デフォルト値を設定
)
content2 = dcc.Dropdown(
    id = 'dropdown-for-plot-1', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in plot_type_1], # 値を選択する列を指定
    value = default_plot_1 # デフォルト値を設定
)
content3 = dcc.Graph(id='box-plot')

グラフ２（基本グラフ）のコンテンツの定義

In [41]:
content4 = dcc.Dropdown(
    id = 'dropdown-for-pref-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in df1['都道府県'].unique()], # 都道府県を指定
    value = default_pref_2 # デフォルト値を設定
)
content5 = dcc.Dropdown(
    id = 'dropdown-for-kpi-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 指標値の種類を指定
    value = default_kpi_2 # デフォルト値を設定
)
content6 = dcc.Dropdown(
    id = 'dropdown-for-plot-2', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in plot_type_2], # グラフのタイプを指定
    value = default_plot_2 # デフォルト値を設定
)
content7 = dcc.Graph(id='basic-plot')

グラフ３（データテーブル）のコンテンツの定義

In [42]:
# 表示する列を定義する
columns1 = [
    dict(id='都道府県',name='都道府県'),
    dict(id='2007年', name='2007年度'),
    dict(id='2008年', name='2008年度'),
    dict(id='2009年', name='2009年度'),
    dict(id='2010年', name='2010年度'),
    dict(id='2011年', name='2011年度'),
    dict(id='2012年', name='2012年度'),
    dict(id='2013年', name='2013年度'),
    dict(id='2014年', name='2014年度'),
    dict(id='2015年', name='2015年度'),
    dict(id='2016年', name='2016年度'),
    dict(id='2017年', name='2017年度'),
    dict(id='2018年', name='2018年度')
]
# ドロップダウンリストを定義する
content8 = dcc.Dropdown(
    id = 'dropdown-for-kpi-3', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 値を選択する列を指定
    value = default_kpi_3 # デフォルト値を設定
)
# データテーブルを定義する
content9 = html.Div(id='datatable-paging',children=[]) # id名はコールバックで必要

画面のレイアウトを設定する

In [43]:
# 画面のレイアウトを定義する
app.layout = dbc.Container([
    dbc.Row([
             dbc.Col(html.H1("日本の都道府県の各種指標",
                             className='text-center text-primary, mb-4'), # 中寄せでタイトルの下にスペース
                     width=12) # 全幅(12列分)の幅をとる
    ]),
    dbc.Row([
             dbc.Col([
                      html.P("各年度の指標値の分布",style={'textDecoration': 'underline'}),
                      html.Div(content1,style={'width':'30%','display':'inline-block','margin-right':10}),
                      html.Div(content2,style={'width':'30%','display':'inline-block','margin-right':10}),
                      content3
             ],width={'size':6, 'order':1}), # 左に余白を設けたい場合は'offset':列数を指定する。orderは図の順序
#             xs=12, sm=12, md=12, lg=5, xl=5), # 小さいサイズの画面の場合は幅を12にして縦に並ぶようにする
            dbc.Col([
                     html.P("都道府県の指標値の推移",style={'textDecoration': 'underline'}),
                     html.Div(content4,style={'width':'30%','display':'inline-block','margin-right':10}),
                     html.Div(content5,style={'width':'30%','display':'inline-block','margin-right':10}),
                     html.Div(content6,style={'width':'30%','display':'inline-block','margin-right':10}),
                     content7
            ],width={'size':6, 'order':1}), # 左に余白を設けたい場合は'offset':列数を指定する。orderは図の順序
#             xs=12, sm=12, md=12, lg=5, xl=5), # 小さいサイズの画面の場合は幅を12にして縦に並ぶようにする)
    ], justify='around', align='center'), 
    dbc.Row([
             dbc.Col([
                    html.P("指標の数値一覧",style={'textDecoration': 'underline'}),
                    html.Div(content8, style={'width': '30%', 'display': 'inline-block','margin-right': 10}),
                    content9      
             ], width={'size':'auto', 'order':1}) # width="auto"はコンテンツの自然な幅
    ] ,justify='around', align='center')
], fluid=True) # fluid=Trueで端の空白をなくす

グラフ１（分布グラフ）のコールバックと実行関数

In [44]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに図を描画する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='box-plot',component_property='figure'),
    [Input(component_id='dropdown-for-kpi-1',component_property='value'),
     Input(component_id='dropdown-for-plot-1',component_property='value')]
)
def generate_chart(inval1,inval2): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  if inval2 == '箱ひげ図':
    fig = go.Figure()
    fig.add_trace(go.Box(x=df1['ch_year'],y=df1[inval1],text=df1['都道府県']))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  elif inval2 == '散布図':
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df1['ch_year'],y=df1[inval1],
                      mode='markers',text=df1['都道府県']))
    fig.update_traces(marker=dict(size=12,line=dict(width=2,color='DarkSlateGray')),
                      selector=dict(mode='markers'))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  else:
    fig = go.Figure()
    fig.add_trace(go.Violin(x=df1['ch_year'],y=df1[inval1],text=df1['都道府県']))
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":16})
  return fig

グラフ２（基本グラフ）のコールバックと実行関数

In [45]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに図を描画する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='basic-plot',component_property='figure'),
    [Input(component_id='dropdown-for-pref-2',component_property='value'),
     Input(component_id='dropdown-for-kpi-2',component_property='value'),
     Input(component_id='dropdown-for-plot-2',component_property='value')]
)
def generate_chart(inval1,inval2,inval3): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  df201 = df1[df1['都道府県']==inval1] # ドロップダウンリストで指定された都道府県に絞り込む
  if inval3 == '折れ線グラフ':
    fig = px.line(df201, x='年度', y=inval2) # 折れ線グラフを描画する
    fig.update_layout(height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
  else:
    fig = px.bar(df201, x='年度', y=inval2) # 棒グラフを描画する
    fig.update_layout(height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
  return fig

グラフ３（データテーブル）のコールバックと実行関数

In [46]:
# データテーブルを描画する関数を定義する
# 一行おきに色のコントラストをつけたデザインにする
def create_dash_table(df):
  return dash_table.DataTable(
      data=df.to_dict('records'),
      columns=columns1,
      style_cell={'fontsize':20, 'font-family':'IPAexGothic'},
      style_cell_conditional=[
                              {
                                  'if': {'column_id':c},
                                  'textAlign': 'left'
                              } for c in ['都道府県']
      ],
      style_data={
      'color':'black',
      'backgroundColor':'white'
      },
      style_data_conditional=[
                          {
                              'if': {'row_index':'odd'},
                              'backgroundColor': 'rgb(220,220,220)',
                          }
      ],
      style_header={
          'backgroundColor':'rgb(210,210,210)',
          'color':'black',
          'fontWeight':'bold'
  })

In [47]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに表を作成する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='datatable-paging',component_property='children'),
    [Input(component_id='dropdown-for-kpi-3',component_property='value')]
)
def update_table(inval1): # 行間をあけずに関数部分を書く（関数名は何でも良い）
    df502 = df1.pivot(index='id_pref', columns='ch_year', values=inval1)
    df502.reset_index(inplace=True)
    df502 = df502.assign(都道府県=df502['id_pref'].apply(lambda x: x[6:]))
    return create_dash_table(df502)

In [48]:
# サーバーを起動する
if __name__ == '__main__':
    app.run_server(debug=True, port=3000)

Dash app running on:


<IPython.core.display.Javascript object>